In [1]:
!pip install -qU \
    nemoguardrails==0.4.0 \
    openai==0.27.8

We need to set our OpenAI API key:

In [2]:
import os

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or "sk-..."

We can run Guardrails from terminal using the command:

```
nemoguardrails chat --config=config/
```

Where the `config` directory must contain our `config.yml` and a Colang file (like `topics.co`).

Alternatively we can load them from file using `RailsConfig.from_path("./config")` or from string variables in our code like so:

In [3]:
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
"""
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [4]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
# create rails
rails = LLMRails(config)

From here, we begin asking questions and interacting with our Guardrails protected LLM.

In [5]:
res = await rails.generate_async(prompt="Hey there!")
print(res)

Hi there! How can I help you?
How are you doing today?


This is a typical greeting so we have no protective guardrails here. However, we do see the `greeting` flow being activated as the chatbot generates some text from the `bot express greeting` message, and on the next line generates some more text from the `bot ask how are you` message.

Let's try asking a more political question:

In [6]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)

I'm a shopping assistant, I don't like to talk of politics.
However, I can help you with shopping related tasks. Is there anything I can help you with?


Here we can see that our `politics` rail is activated and our chatbot dodges the question with `bot answer politics` and follows up with `bot offer help`.

---